In [136]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from gorbslam.common.utils import to_tum
from gorbslam.common.plotting_utils import create_ape_fig


from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'vabadussild'
TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'

# model_type = ModelType.NN
# model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()


Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[P

Best BGR hyperparameters: {'estimator__max_depth': 30, 'estimator__min_samples_leaf': 4, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 10}


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Do

In [138]:
processor.fit_trajectories()


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

In [139]:
processor.save_trajectories()


In [140]:
fig_2d = processor.create_2d_plot_utm()
fig_2d.show()


In [141]:
fig_slam = processor.create_2d_plot_slam()
fig_slam.show()


In [142]:
fig_map = processor.create_map_plot()
fig_map.show()


In [143]:
trajs = {
    "fitted_slam": (
        to_tum(
            processor.orbslam.mapping._trajectory_tum,
            processor.orbslam.mapping.trajectory_fitted_utm,
        ),
        to_tum(
            processor.orbslam.mapping._trajectory_tum_gt_wgs,
            processor.orbslam.mapping.trajectory_gt_utm,
        ),
    ),
    "fitted_loc_0": (
        to_tum(
            processor.orbslam.localization[0]._trajectory_tum,
            processor.orbslam.localization[0].trajectory_fitted_utm,
        ),
        to_tum(
            processor.orbslam.localization[0]._trajectory_tum_gt_wgs,
            processor.orbslam.localization[0].trajectory_gt_utm,
        ),
    ),
    # 'fitted_loc_1':(to_tum(processor.orbslam.localization[1]._trajectory_tum, processor.orbslam.localization[1].trajectory_fitted_utm), to_tum(processor.orbslam.localization[1]._trajectory_tum_gt_wgs, processor.orbslam.localization[1].trajectory_gt_utm)),
    # "fitted_loc_2": (
    #     to_tum(
    #         processor.orbslam.localization[2]._trajectory_tum,
    #         processor.orbslam.localization[2].trajectory_fitted_utm,
    #     ),
    #     to_tum(
    #         processor.orbslam.localization[2]._trajectory_tum_gt_wgs,
    #         processor.orbslam.localization[2].trajectory_gt_utm,
    #     ),
    # ),
    # 'fitted_loc_3':(to_tum(processor.orbslam.localization[3]._trajectory_tum, processor.orbslam.localization[3].trajectory_fitted_utm), to_tum(processor.orbslam.localization[3]._trajectory_tum_gt_wgs, processor.orbslam.localization[3].trajectory_gt_utm)),
}

# name = 'fitted_slam'
name = "fitted_loc_0"
# name = "fitted_loc_1"
# name = "fitted_loc_2"
# name = 'fitted_loc_3'

print(trajs[name][0].shape)

ape_fig = create_ape_fig(*trajs[name])
ape_fig.show()


(2926, 8)
Min APE: 0.005440786876620142
Max APE: 6.8741990781116575
